In [5]:
from gio import GIO
import numpy as np

##Data from Chan et al. (2018)'s Example 1##
#Notice that we have numpy arrays here.
A = np.array([[2,5],[2,-3],[2,1],[-2,-1]])
b = np.array([[10],[-6],[4],[-10]])
x0 = np.array([[2.5],[3]])

##Creating the GIO object and Generating all of the x^0-ep* for all GIO Models## 
#.GIO_all_measures() calls all of the GIO methods
gio_testing = GIO(A,b,x0)
gio_testing.GIO_all_measures()
print("This is x^0-ep* for p=1: ",gio_testing.x0_epsilon_p[0])
print("This is x^0-ep* for p=2: ",gio_testing.x0_epsilon_p[1])
print("This is x^0-ep* for p='inf': ",gio_testing.x0_epsilon_p[2])
print("This is x^0-ep* for absolute duality: ",gio_testing.x0_epsilon_a)
print("This is x^0-ep* for relative duality: ",gio_testing.x0_epsilon_r)

This is x^0-ep* for p=1:  [[ 2.5       ]
 [ 3.66666667]]
This is x^0-ep* for p=2:  [[ 2.19230769]
 [ 3.46153846]]
This is x^0-ep* for p='inf':  [[ 2.1]
 [ 3.4]]
This is x^0-ep* for absolute duality:  [array([[ 2.1],
       [ 3.4]])]
This is x^0-ep* for relative duality:  [array([[ 3.16666667],
       [ 3.66666667]])]


In [6]:
##Because we are storing the calculated attributes in lists, we see that, 
##for the last two print statements in the previous cell, a list was returned 
##with an array inside of it.  We can solve this problem by add an index element [0] 
print("This is x^0-ep* for absolute duality: ",gio_testing.x0_epsilon_a[0])
print("This is x^0-ep* for relative duality: ",gio_testing.x0_epsilon_r[0])

This is x^0-ep* for absolute duality:  [[ 2.1]
 [ 3.4]]
This is x^0-ep* for relative duality:  [[ 3.16666667]
 [ 3.66666667]]


In [ ]:
##The GIO class also handles cases when you might minimally project onto multiple
##hyperplanes. The class will put the multiple istar indices into the .istar_multi 
##attribute but will choose the first istar index as the one to continue calculations.

##The class will also output a message to the user to let the user know that 
##this has occurred. 

A_1 = np.array([[-1,-1],[1,-1],[1,1],[-1,1]]) 
b_1 = np.array([[-1],[-1],[-1],[-1]])
x0_1 = np.array([[0],[0.7]])


The corresponding feasible region set up for the above $A$ and $b$ is as follows:

\begin{equation}
-x_1 - x_2 \geq -1
\end{equation}
\begin{equation}
x_1 - x_2 \geq -1
\end{equation}
\begin{equation}
x_1 + x_2 \geq -1
\end{equation}
\begin{equation}
-x_1 + x_2 \geq -1
\end{equation}

We can then demonstrate the feasible region using the image below (with approximate placement of $x^0$ below). The numbering corresponds to the order of the constraints below (and then the 0 indexing of Python).

![alt text](Steph_Test_Ex.jpg "Feasible Region")

In [7]:
##We will create another GIO object to find the optimal epsilon* under the p=1 norm.
##Due to the position of x0_1 and the properties of the p=1 norm, we know that this
##will result into a projection onto multiple hyperplanes

#We will need to bring in everything again because we had a break in the code 
#cells (with the markup cell above)

from gio import GIO
import numpy as np

A_1 = np.array([[-1,-1],[1,-1],[1,1],[-1,1]]) 
b_1 = np.array([[-1],[-1],[-1],[-1]])
x0_1 = np.array([[0],[0.7]])

gio_multi_project_testing = GIO(A_1,b_1,x0_1)
gio_multi_project_testing.GIO_p(1,'F') 

Under the inf dual norm, x^0 has been projected onto multiple hyperplanes.  For now, we will choose the first i index and will put the rest of the indices in the istar_multi attribute.


In [8]:
##As we can see, a message is printed to the user when x^0 is projected onto multiple
##hyperplanes.
##We can see the multiple istar if we print the .istar_multi attribute
print(gio_multi_project_testing.istar_multi)

[array([0, 1], dtype=int64)]


In [9]:
##To obtain the raw indices, index into the list
print("Raw Indices: ", gio_multi_project_testing.istar_multi[0])
##To obtain only one of the indices, do a double index
print("Single index: ",gio_multi_project_testing.istar_multi[0][0])

Raw Indices:  [0 1]
Single index:  0


In [10]:
##We see though that only one epsilon* is calculated and thus only one x^0-epsilon* 
##is also calculated
print("This is the epsilon* for p=1: ",gio_multi_project_testing.epsilon_p)
print("This is the (x^0 - epsilon*): ",gio_multi_project_testing.x0_epsilon_p)

This is the epsilon* for p=1:  [array([[-0.3],
       [-0. ]])]
This is the (x^0 - epsilon*):  [array([[ 0.3],
       [ 0.7]])]


In [5]:
###Fiddling Around with Pyomo Objects###
#Need to figure out what the Parameter A looks like
#MAYBE LATER: We can look into if we can recover data from a model
#that is set up well for the forward problem but that does not 
#define this A
import pyomo.environ as pyo
import numpy as np

A = np.array([[1,2],[3,4]])
b = np.array([[1],[2]])

test_model = pyo.ConcreteModel()
test_model.varindex = pyo.RangeSet(2)
test_model.numvars = pyo.Param(initialize=2)
test_model.x = pyo.Var(test_model.varindex)
def A_mat_func(model,i,j):
    return A[i-1,j-1]

test_model.Amat = pyo.Param(test_model.varindex,test_model.varindex,rule=A_mat_func)

def constraint_rule(model,i):
    return sum( model.Amat[i,j]*model.x[j] for j in range(1,model.numvars+1)) <= b[i-1,:]

test_model.constraints = pyo.Constraint(test_model.varindex,rule=constraint_rule)

print("This is the number of variables:",test_model.numvars.value)

print("This is the model:")
test_model.pprint()

print("This is just the parameter A component/attribute of the model")
print(test_model.Amat[1,1]) #doesnt look like I can use the slicer operation 
                            #(tried and failed)
                            #so I think I'm going to have to have a function that
                            #basically iterates through the "A" parameter matrix
                            #and puts into a numpy array - will have that in the initialization
                            #set up
                        
                            #We can easily unittest this as well. (Just literally put example 1
                            #in a pyomo model with Amat and bvec and runs through the same unit test)

A_test = test_model.Amat

print("Trying......",A_test[1,2])

type(A_test)


    


This is the number of variables: 2
This is the model:
1 Set Declarations
    Amat_index : Dim=0, Dimen=2, Size=4, Domain=None, Ordered=True, Bounds=None
        Virtual

1 RangeSet Declarations
    varindex : Dim=0, Dimen=1, Size=2, Domain=Integers, Ordered=True, Bounds=(1, 2)
        Virtual

2 Param Declarations
    Amat : Size=4, Index=Amat_index, Domain=Any, Default=None, Mutable=False
        Key    : Value
        (1, 1) :     1
        (1, 2) :     2
        (2, 1) :     3
        (2, 2) :     4
    numvars : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :     2

1 Var Declarations
    x : Size=2, Index=varindex
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :  None :  None :  None : False :  True :  Reals
          2 :  None :  None :  None : False :  True :  Reals

1 Constraint Declarations
    constraints : Size=2, Index=varindex, Active=True
        Key : Lower : Body            : Upper : Active
      

pyomo.core.base.param.IndexedParam